# Demonstration of shifty.py's "loader" module
 - 20191106
 - Payne 
 
The loader module facilitates loading fits files from disk into an "ImageDataSet" object
 - To do this requires not only simple loading, but also more detailed "cleaning" and "preparation" of the data to ensure that the ImageDataSet object that is ultimately created has only the required, clean data in it. 
 - The ImageDataSet object is described in more detail elsewhere (e.g. demo_data.ipynb) 

There is a "parent" class" called "ImageLoader".
 - This is NOT likely to be used directly at all 

There are "child" classes for each type of data that will be downloaded:
 - These ARE likely to be directly used in practice. 
 
It intended that there be a specific loader for a specific kind of data.
E.g.
 - The "TESSImageLoader" class provides methods to download TESS data (from mast)
 - The "HSTImageLoader" class is just a stub and does not yet have much functionality
 
This notebook:
 - Demonstrates the key components / classes / functions available in the loader module
 - Acts as a complement to the tests of the same module. 

#### Import statements 

In [7]:
# -------------------------------------------------------------------------------------
# Third party imports
# -------------------------------------------------------------------------------------
import numpy as np 
import matplotlib.pyplot as plt
import os, sys
import glob 

import astropy
from astropy.io import fits
from astropy.time import Time
from astropy.units import allclose as quantity_allclose
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates.builtin_frames import FK5, ICRS, GCRS, GeocentricMeanEcliptic, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, GeocentricTrueEcliptic, BarycentricTrueEcliptic, HeliocentricTrueEcliptic, HeliocentricEclipticIAU76
from astropy.constants import R_sun, R_earth

import importlib

# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------
import data 
import loader
import downloader 

importlib.reload(data)
importlib.reload(loader)
importlib.reload(downloader)



<module 'loader' from '/Users/matthewjohnpayne/Envs/shifty/shifty/loader.py'>

### Instantiate TESSImageLoader object
 - Note that TESSImageLoader object has an 'obs_code' quantity: this is passed into the output ImageDataSet to allow later understanding/calculation of the location of the observatory as a function of time. 

In [8]:
# creation of TESSImageLoader object
T = loader.TESSImageLoader()

# examination of instantiated variables
# - these are defining some local data storage 
for k,v in T.__dict__.items():
    print(k,v)
print()

# check that storage directories exist on disk 
for d in [T.local_dir, T.tess_dir ]:
    print( 'It is *%r* that %r exists on disk' % (os.path.isdir(d), d)  )

obs_code C57
local_dir /Users/matthewjohnpayne/.shifty_data
tess_dir /Users/matthewjohnpayne/.shifty_data/tess

It is *True* that '/Users/matthewjohnpayne/.shifty_data' exists on disk
It is *True* that '/Users/matthewjohnpayne/.shifty_data/tess' exists on disk


### Use TESSImageLoader to generate an ImageDataSet based on just a small set of development data
 - Note that this just loads 10 fits files (this is specified below by passing in development = True)
 - Note that this does *no* cleaning (the default is to do no cleaning)  

In [9]:
# load the test data(this is specified below by passing in development = True)
IDU = T.get_image_data_set( **{'development' : True} )

# check that the returned object actually is an ImageDataSet 
print('It is %r that the returned object is an %r' % (isinstance(IDU, data.ImageDataSet), data.ImageDataSet) )

# display some of the contents ...
for k,v in IDU.__dict__.items():
    print(k,v, '\n')

It is True that the returned object is an <class 'data.ImageDataSet'>
mid_times [2458410.9329035  2458410.95373709 2458410.97457068 2458410.99540427
 2458411.01623785 2458411.03707144 2458411.05790502 2458411.0787386
 2458411.09957218 2458411.12040575] 

data [[[ 0.18281528  0.07671411  0.1595356  ...  0.00712733  0.01205946
    0.01674499]
  [ 0.18610774  0.1343039   0.20602259 ...  0.04289078  0.01402684
   -0.02938142]
  [ 0.13516839  0.11913346  0.093174   ...  0.00564221 -0.01113253
   -0.03579868]
  ...
  [ 0.04355072  0.09161711  0.06145985 ... -0.0552806  -0.07920694
   -0.02321625]
  [ 0.06528474  0.06574507  0.03978013 ... -0.14580718 -0.11916818
   -0.07846712]
  [ 0.09491035  0.04110625  0.05460934 ... -0.07082231 -0.10658576
   -0.05404757]]

 [[ 0.16625361  0.08259915  0.12052725 ...  0.0146773   0.01960943
    0.02429496]
  [ 0.16953827  0.14018112  0.12211307 ...  0.07288744 -0.00086988
   -0.04427814]
  [ 0.20837788  0.10255622  0.07659676 ...  0.01319218 -0.04847594
 

### Use TESSImageLoader to load prf's (previously downloaded using the TESSDownloader module)

In [10]:
listHDUs = T.get_prfs()
prf_filepaths = T.download_prf()

len(listHDUs), prf_filepaths

(0,
 ['/Users/matthewjohnpayne/.shifty_data/prf/start_s0001/tess_prf_camera_1.tgz',
  '/Users/matthewjohnpayne/.shifty_data/prf/start_s0001/tess_prf_camera_2.tgz',
  '/Users/matthewjohnpayne/.shifty_data/prf/start_s0001/tess_prf_camera_3.tgz',
  '/Users/matthewjohnpayne/.shifty_data/prf/start_s0001/tess_prf_camera_4.tgz',
  '/Users/matthewjohnpayne/.shifty_data/prf/start_s0004/tess_prf_camera_1.tgz',
  '/Users/matthewjohnpayne/.shifty_data/prf/start_s0004/tess_prf_camera_2.tgz',
  '/Users/matthewjohnpayne/.shifty_data/prf/start_s0004/tess_prf_camera_3.tgz',
  '/Users/matthewjohnpayne/.shifty_data/prf/start_s0004/tess_prf_camera_4.tgz'])

### Use TESSImageLoader to perform various "cleaning" tasks ... 

#### masking stars
 - It be of use to simply 'mask-out' the regions/pixels on an image that have stars on them. 
 - Here we pass a mask=True option to the *get_image_data_set()* function to turn-on the masking
 - Note that this is ultimately calling the _mask_stars() function to do the work of masking

In [18]:
IDU = T.get_image_data_set( **{'development' : True , 'mask' : True } )
print(' * AT PRESENT THIS DOES NOTHING BECAUSE THE MASK FUNCTION JUST PASSES THE DATA THROUGH WITHOUT DOING ANYTHING !!!! * ')

 * AT PRESENT THIS DOES NOTHING BECAUSE THE MASK FUNCTION JUST PASSES THE DATA THROUGH WITHOUT DOING ANYTHING !!!! * 
